This program will be an example to showcase how to use the edc522, keithley193a (both non SCPI instruments) as well as the arduino model and showcase how to make a rudimentary program

In [1]:
from piec.drivers.utilities import PiecManager
from piec.drivers.arduino import Arduino_Stepper
from piec.drivers.edc522 import EDC522
from piec.drivers.keithley193a import Keithley193a
pm = PiecManager()


In [2]:
pm.list_resources()

('GPIB0::10::INSTR',)

In [3]:
Stepper = Arduino_Stepper("ASRL3::INSTR")
Stepper.idn()

VisaIOError: VI_ERROR_RSRC_NFOUND (-1073807343): Insufficient location information or the requested device or resource is not present in the system.

In [3]:
voltage_program = EDC522("GPIB0::9::INSTR")
voltage_program.idn()

'ID?\r\n000'

In [ ]:
voltage_program.set_output()

TypeError: '>' not supported between instances of 'str' and 'int'

In [16]:
voltage_program.instrument.write("00000000")

10

In [9]:
voltage_program.set_output(1)

In [3]:
voltage_out = Keithley193a("GPIB0::10::INSTR")
voltage_out.idn()   # This will print the IDN of the instrument

'NDCV-0000.003E+0\r\n'

In [7]:
voltage_out.read_voltage()

'-0000.005E+0'